In [ ]:
# imports
from matplotlib import pyplot as plt
import numpy as np
import math

import cupy as cp

from mpl_toolkits.mplot3d import Axes3D

from scipy import signal, misc


# Create - Data, Consts, Pars, Lower Bound, Upper Bound

In [ ]:
from dipy.data import get_fnames
from dipy.io.image import load_nifti_data, save_nifti, load_nifti
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table

fraw,fbval,fbvec = get_fnames('ivim')

_, affine = load_nifti(fraw)
data = np.float32(load_nifti_data(fraw))
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals, bvecs, b0_threshold=0)

data_shape = data.shape
nvoxels = data_shape[0]*data_shape[1]*data_shape[2]
ndata = data_shape[3]

data = np.float32(data.reshape(1, data.size))
consts = np.float32(np.reshape(np.tile(bvals, nvoxels), (1,nvoxels*len(bvals))))

redo_pars_t = False
if redo_pars_t:

    from dipy.reconst.ivim import IvimModel
    ivimmodel = IvimModel(gtab, fit_method='trr')

    ivimfit = ivimmodel.fit(data)

    pars_t = ivimfit.model_params

    save_nifti('pars_dipy.nii', pars_t, affine)

pars_dipy, affine = load_nifti('pars_dipy.nii')
pars_dipy = np.float32(pars_dipy)

lower_bound = np.float32(np.ascontiguousarray(
                np.tile(
                    np.array([np.finfo(np.float32).min / 2, 0.0, 0.0, 0.0]), (data.shape[1],1)).transpose()))

upper_bound = np.float32(np.ascontiguousarray(
                np.tile(
                    np.array([np.finfo(np.float32).max / 2, 1.0, 1.0, 1.0]), (data.shape[1],1)).transpose()))


# Create guess parameters from dipys by convolving

In [ ]:

kernel_size = 25
avg_kernel = np.ones((kernel_size,kernel_size))

pars = np.empty(pars_dipy.shape, pars_dipy.dtype)

for i in range(0,pars.shape[2]):
    for j in range(0,4):
        pars[:,:,i,j] = signal.convolve2d(pars_dipy[:,:,i,j], avg_kernel, boundary='symm', mode='same')


In [ ]:

print_S0 = False
print_f = False
print_D1 = False
print_D2 = True

if print_S0:
	fig1 = plt.figure()
	ax1 = fig1.add_axes([0,0,1,1])
	ax1.imshow(pars_dipy[:,:,25,0])
	ax1.set_title('S0')

	fig2 = plt.figure()
	ax2 = fig2.add_axes([0,0,1,1])
	ax2.imshow(pars[:,:,25,0])
	ax2.set_title('S0 - Avg')

if print_f:
	fig1 = plt.figure()
	ax1 = fig1.add_axes([0,0,1,1])
	ax1.imshow(pars_dipy[:,:,25,1])
	ax1.set_title('f')

	fig2 = plt.figure()
	ax2 = fig2.add_axes([0,0,1,1])
	ax2.imshow(pars[:,:,25,1])
	ax2.set_title('f - Avg')

if print_D1:
	fig1 = plt.figure()
	ax1 = fig1.add_axes([0,0,1,1])
	ax1.imshow(pars_dipy[:,:,25,2])
	ax1.set_title('D1')

	fig2 = plt.figure()
	ax2 = fig2.add_axes([0,0,1,2])
	ax2.imshow(pars[:,:,25,2])
	ax2.set_title('D1 - Avg')

if print_D2:
	fig1 = plt.figure()
	ax1 = fig1.add_axes([0,0,1,1])
	ax1.imshow(pars_dipy[:,:,25,3])
	ax1.set_title('D2')

	fig2 = plt.figure()
	ax2 = fig2.add_axes([0,0,1,2])
	ax2.imshow(pars[:,:,25,3])
	ax2.set_title('D2 - Avg')


In [ ]:
pars = np.reshape(pars, (4, nvoxels))

In [ ]:
from cuda.lsqnonlin import SecondOrderLevenbergMarquardt

expr = 'S0*(f*exp(-b*D_1)+(1-f)*exp(-b*D_2))'
pars_str = ['S0', 'f', 'D_1', 'D_2']
consts_str = ['b']

nchunks = 50
data_chunk_size = math.ceil(data.shape[1] / nchunks)


for i in range(0,1):

    parscu = cp.asarray(pars[:,i*data_chunk_size:(i+1)*data_chunk_size])
    constscu = cp.asarray(data[:,i*data_chunk_size*ndata:(i+1)*data_chunk_size*ndata])
    datacu = cp.asarray(data[:,i*data_chunk_size*ndata:(i+1)*data_chunk_size*ndata])
    lower_bound_cu = cp.asarray(lower_bound[:,i*data_chunk_size:(i+1)*data_chunk_size])
    upper_bound_cu = cp.asarray(upper_bound[:,i*data_chunk_size:(i+1)*data_chunk_size])

    solm = SecondOrderLevenbergMarquardt(expr, pars_str, consts_str, parscu, constscu, datacu, lower_bound_cu, upper_bound_cu)
    solm.run(10, 1e-5)
